<a href="https://colab.research.google.com/github/BhumitJoshi/5-Sem-CEC-Term-Work/blob/main/Rama_Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies

In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [20]:
# loading the dataset to a Pandas DataFrame
credit_card_data = pd.read_csv('/content/credit_data.csv')

In [21]:
# first 5 rows of the dataset
credit_card_data.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
180346,124501.0,1.776084,-0.847941,-0.122495,0.327068,-0.901450,0.297088,-1.002304,0.351652,1.498805,...,0.158905,0.350790,0.255732,0.656765,-0.493054,-0.462741,0.029698,-0.019629,79.55,0.0
180347,124501.0,2.112579,-0.978592,-2.531591,-1.672016,1.621659,3.410626,-1.522773,0.940170,-0.095966,...,0.241275,0.658909,0.137661,0.604387,-0.114413,-0.067851,0.047220,-0.016706,24.99,0.0
180348,124501.0,1.964991,-0.268471,-1.385679,0.273598,-0.092173,-1.228189,0.269715,-0.398833,0.388609,...,0.249128,0.721101,-0.008697,-0.000683,0.033707,0.787556,-0.097355,-0.060784,64.95,0.0
180349,124501.0,-1.538327,1.432749,-0.122378,-2.499628,-0.198887,-1.405793,0.459504,0.794566,-0.200864,...,-0.179255,-0.728338,0.009726,0.055995,-0.063253,0.358524,0.075288,0.086458,4.00,0.0
180350,124501.0,1.954788,-0.405739,-0.322175,0.333503,-0.540118,-0.094393,-0.651349,0.055564,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# dataset informations
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180351 entries, 0 to 180350
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    180351 non-null  float64
 1   V1      180351 non-null  float64
 2   V2      180351 non-null  float64
 3   V3      180351 non-null  float64
 4   V4      180351 non-null  float64
 5   V5      180351 non-null  float64
 6   V6      180351 non-null  float64
 7   V7      180351 non-null  float64
 8   V8      180351 non-null  float64
 9   V9      180350 non-null  float64
 10  V10     180350 non-null  float64
 11  V11     180350 non-null  float64
 12  V12     180350 non-null  float64
 13  V13     180350 non-null  float64
 14  V14     180350 non-null  float64
 15  V15     180350 non-null  float64
 16  V16     180350 non-null  float64
 17  V17     180350 non-null  float64
 18  V18     180350 non-null  float64
 19  V19     180350 non-null  float64
 20  V20     180350 non-null  float64
 21  V21     18

In [23]:
# checking the number of missing values in each column
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        1
V10       1
V11       1
V12       1
V13       1
V14       1
V15       1
V16       1
V17       1
V18       1
V19       1
V20       1
V21       1
V22       1
V23       1
V24       1
V25       1
V26       1
V27       1
V28       1
Amount    1
Class     1
dtype: int64

In [24]:
credit_card_data.dropna(inplace=True)

In [25]:
# checking the number of missing values in each column
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [26]:
# distribution of legit transactions & fraudulent transactions
credit_card_data['Class'].value_counts()

0.0    179986
1.0       364
Name: Class, dtype: int64

This Dataset is highly unblanced

0 --> Normal Transaction

1 --> fraudulent transaction

In [27]:
# separating the data for analysis
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [28]:
print(legit.shape)
print(fraud.shape)

(179986, 31)
(364, 31)


In [29]:
# statistical measures of the data
legit.Amount.describe()

count    179986.00000
mean         88.19112
std         246.98978
min           0.00000
25%           5.74000
50%          22.24000
75%          77.93000
max       19656.53000
Name: Amount, dtype: float64

In [30]:
fraud.Amount.describe()

count     364.000000
mean      116.743462
std       249.586886
min         0.000000
25%         1.000000
50%        11.395000
75%       104.030000
max      2125.870000
Name: Amount, dtype: float64

In [31]:
# compare the values for both transactions
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0.0,64325.261476,-0.133518,0.016479,0.446990,0.089890,-0.146387,0.057177,-0.057269,0.025063,0.021303,...,0.026267,-0.024279,-0.070453,-0.020042,0.007367,0.080854,0.010345,0.001693,0.002074,88.191120
1.0,57374.343407,-5.992428,4.358422,-7.940295,4.868038,-4.344519,-1.432647,-6.964458,0.747432,-2.847610,...,0.411224,0.796613,-0.019094,-0.042130,-0.072836,0.077923,0.036389,0.178167,0.048976,116.743462


Under-Sampling

Build a sample dataset containing similar distribution of normal transactions and Fraudulent Transactions

Number of Fraudulent Transactions --> 492

In [32]:
legit_sample = legit.sample(n=364)

Concatenating two DataFrames

In [33]:
new_dataset = pd.concat([legit_sample, fraud], axis=0)

In [34]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
144961,86521.0,1.996436,0.084790,-0.648046,1.715559,0.138153,-0.023723,-0.173426,-0.056777,-0.897585,...,-0.052020,-0.075596,0.268775,-0.280403,-0.576572,2.145696,-0.186738,-0.089539,0.00,0.0
106990,70223.0,1.102457,0.133914,0.385561,0.680534,-0.339168,-0.822133,0.271146,-0.152886,-0.137486,...,-0.419905,-1.339846,0.283169,0.360994,-0.031321,-0.123003,-0.034268,0.027341,54.49,0.0
100231,67458.0,-1.171967,-2.186156,2.427245,-0.753041,-1.200884,0.251946,-1.620317,0.237675,-0.784193,...,-0.169946,0.530244,0.589038,0.088958,-0.292944,-0.017302,0.208374,0.203828,67.30,0.0
19626,30420.0,-0.395203,1.135149,1.323384,0.071842,0.210376,-1.010067,0.732631,-0.132658,-0.565801,...,-0.244150,-0.597841,-0.015484,0.366180,-0.117832,0.067977,0.250735,0.102370,2.28,0.0
91603,63552.0,1.158543,-0.852010,0.253902,-0.551209,-0.809348,-0.061647,-0.490408,0.026213,-0.853973,...,0.186989,0.480298,-0.165661,0.077533,0.568639,-0.143476,0.005892,0.010179,92.70,0.0


In [35]:
new_dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
167305,118603.0,-6.677212,5.529299,-7.193275,6.081321,-1.636071,0.500610,-4.640770,-4.339840,-0.950036,...,5.563301,-1.608272,0.965322,0.163718,0.047531,0.466165,0.278547,1.471988,105.89,1.0
172787,121238.0,-2.628922,2.275636,-3.745369,1.226948,-1.132966,-1.256353,-1.752420,0.281736,-1.792343,...,0.870730,1.269473,-0.265494,-0.480549,0.169665,0.096081,0.070036,0.063768,144.62,1.0
176049,122608.0,-2.003460,-7.159042,-4.050976,1.309580,-2.058102,-0.098621,2.880083,-0.727484,1.460381,...,1.244287,-1.015232,-1.800985,0.657586,-0.435617,-0.894509,-0.397557,0.314262,2125.87,1.0
177195,123078.0,-1.073820,0.415616,-2.273977,1.536844,-0.758697,-1.670381,-2.377140,0.090370,0.004847,...,0.535542,0.863592,0.450743,-0.144228,-0.205609,-0.539073,0.503418,-0.237807,11.00,1.0
178208,123525.0,-5.904921,4.439911,-8.631802,7.788684,-4.989580,-1.200144,-7.674060,4.125761,-5.315778,...,1.775030,1.266441,-0.199410,0.014960,-0.000019,0.544210,-1.167566,-1.206354,45.51,1.0


In [36]:
new_dataset['Class'].value_counts()

0.0    364
1.0    364
Name: Class, dtype: int64

In [37]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0.0,64327.489011,-0.086310,-0.140778,0.512264,0.141350,-0.244420,-0.007947,0.006480,0.029874,0.105425,...,0.020670,-0.015119,-0.018754,-0.023423,0.021611,0.059170,0.026962,-0.001227,0.010343,112.775604
1.0,57374.343407,-5.992428,4.358422,-7.940295,4.868038,-4.344519,-1.432647,-6.964458,0.747432,-2.847610,...,0.411224,0.796613,-0.019094,-0.042130,-0.072836,0.077923,0.036389,0.178167,0.048976,116.743462


Splitting the data into Features & Targets

In [38]:
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']

In [39]:
print(X)

            Time        V1        V2        V3        V4        V5        V6  \
144961   86521.0  1.996436  0.084790 -0.648046  1.715559  0.138153 -0.023723   
106990   70223.0  1.102457  0.133914  0.385561  0.680534 -0.339168 -0.822133   
100231   67458.0 -1.171967 -2.186156  2.427245 -0.753041 -1.200884  0.251946   
19626    30420.0 -0.395203  1.135149  1.323384  0.071842  0.210376 -1.010067   
91603    63552.0  1.158543 -0.852010  0.253902 -0.551209 -0.809348 -0.061647   
...          ...       ...       ...       ...       ...       ...       ...   
167305  118603.0 -6.677212  5.529299 -7.193275  6.081321 -1.636071  0.500610   
172787  121238.0 -2.628922  2.275636 -3.745369  1.226948 -1.132966 -1.256353   
176049  122608.0 -2.003460 -7.159042 -4.050976  1.309580 -2.058102 -0.098621   
177195  123078.0 -1.073820  0.415616 -2.273977  1.536844 -0.758697 -1.670381   
178208  123525.0 -5.904921  4.439911 -8.631802  7.788684 -4.989580 -1.200144   

              V7        V8        V9  .

In [40]:
print(Y)

144961    0.0
106990    0.0
100231    0.0
19626     0.0
91603     0.0
         ... 
167305    1.0
172787    1.0
176049    1.0
177195    1.0
178208    1.0
Name: Class, Length: 728, dtype: float64


Split the data into Training data & Testing Data

In [41]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [42]:
print(X.shape, X_train.shape, X_test.shape)

(728, 30) (582, 30) (146, 30)


Model Training

Logistic Regression

In [43]:
model = LogisticRegression()

In [ ]:
# training the Logistic Regression Model with Training Data
model.fit(X_train, Y_train)

Model Evaluation

In [45]:
# accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [46]:
print('Accuracy on Training data : ', training_data_accuracy)

Accuracy on Training data :  0.9450171821305842


In [47]:
# accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [48]:
print('Accuracy score on Test Data : ', test_data_accuracy)

Accuracy score on Test Data :  0.9452054794520548
